In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import json, requests

StatementMeta(, f5423d2a-a33a-4487-b8af-18ba9cf56016, 3, Finished, Available)

In [ ]:
def AuthenticateWithSP():
    key_vault = ""
    tenant = ""
    client = ""
    client_secret = mssparkutils.credentials.getSecret(key_vault , "clientsecret")    
 
    try:
        from azure.identity import ClientSecretCredential
    except Exception:
        !pip install azure.identity
        from azure.identity import ClientSecretCredential
 
    # Generates the access token for the Service Principal
    api = 'https://graph.microsoft.com/.default'
    auth = ClientSecretCredential(authority = 'https://login.microsoftonline.com/',
                              tenant_id = tenant,
                              client_id = client,
                              client_secret = client_secret)
    access_token = auth.get_token(api)
    access_token = access_token.token
    header = {'Authorization': f'Bearer {access_token}'}
    print('\nSuccessfully authenticated.')
    return header


StatementMeta(, f5423d2a-a33a-4487-b8af-18ba9cf56016, 5, Finished, Available)

In [ ]:
header= AuthenticateWithSP()

StatementMeta(, f5423d2a-a33a-4487-b8af-18ba9cf56016, 6, Finished, Available)


Successfully authenticated.


In [ ]:
schema = StructType(
    [
        StructField('userPrincipalName', StringType(), True),
        StructField('companyName', StringType(), True)
    ]
    )
 
base_url = 'https://graph.microsoft.com/v1.0'
API_url = base_url+"/users?$select=userPrincipalName,companyName"
response = requests.get(API_url, headers=header)
print(response)
if response != None and response.status_code == 200:
    JSONContent = json.loads(response.content)
    JSONContentExplode = JSONContent['value']
   
    JSONContentExplodeNext = JSONContent['@odata.nextLink']
    while JSONContentExplodeNext != None:
        response = requests.get(JSONContentExplodeNext, headers=header)
        if response != None and response.status_code == 200:
            JSONContent = json.loads(response.content)
            JSONContentExplode = JSONContentExplode + JSONContent['value']
 
            if '@odata.nextLink' in JSONContent:
                JSONContentExplodeNext = JSONContent['@odata.nextLink']                
            else:
                JSONContentExplodeNext = None            
        else:
            JSONContentExplodeNext = None
 
    df_JSONContentExplode=spark.createDataFrame(data=JSONContentExplode, schema=schema)      

StatementMeta(, f5423d2a-a33a-4487-b8af-18ba9cf56016, 7, Finished, Available)

In [ ]:
df_JSONContentExplode.write.format('delta')\
.mode("overwrite")\
.option("overwriteSchema", "True")\
.save("Files/AD_users")

StatementMeta(, 2f838cb2-689f-4d7c-90f9-f0c6e2543ce6, 7, Finished, Available)